# Compute electric mix for all countries

The main pipeline of `ecodynelec` presented in tutorials [1](https://ecodynelec.readthedocs.io/en/latest/examples/with_python.html#execution) and [2](https://ecodynelec.readthedocs.io/en/latest/examples/with_spreadsheet.html#execution) only compute the electricity mix and impacts for one target country.

However `ecodynelec` can compute the <u>electric mix</u> for all considered countries, ***all at once***. In facts, this information is an intermediate result in the main pipeline. The feature of extracting this intermediate result was added due to its potential usefulness. This tutorial shows how.

In technical words, a function `get_inverted_matrix` allow to retrieve the invert of technology matrix $(I-A)^{-1}$.

In [1]:
from ecodynelec.pipelines import get_inverted_matrix # Import the function
from ecodynelec.parameter import Parameter # Import the parameter handler object

In [2]:
# Configuration to reduce the number of time steps
my_config = Parameter().from_excel("./Spreadsheet_example.xlsx")
my_config.start="2017-02-01 12:00"
my_config.end="2017-02-01 14:00" # Only 2 hours
my_config.freq = "H" # Only in hourly time step
my_config.path.generation = "./test_data/generations/" # Generation files
my_config.path.exchanges = "./test_data/exchanges/" # Exchanges files

In [3]:
# Execute the function
mix = get_inverted_matrix(config=my_config)

The result is **one matrix per time step**, all stored in a `list`. Here is an overview of the first matrix, i.e. corresponding to the first time step.

In [4]:
display(mix[0])

,Mix_AT,Mix_CH,Mix_DE,Mix_FR,Mix_IT,Mix_Other,Biomass_AT,Fossil_Brown_coal/Lignite_AT,Fossil_Coal-derived_gas_AT,Fossil_Gas_AT,...,Hydro_Run-of-river_and_poundage_IT,Hydro_Water_Reservoir_IT,Marine_IT,Nuclear_IT,Other_fossil_IT,Other_renewable_IT,Solar_IT,Waste_IT,Wind_Offshore_IT,Wind_Onshore_IT
Mix_AT,1.000053,2.964993e-02,2.145851e-04,0.0,0.000990,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mix_CH,0.001788,1.003252e+00,7.260827e-03,0.0,0.033487,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mix_DE,0.224020,4.487106e-01,1.003247e+00,0.0,0.014977,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mix_FR,0.007299,2.315305e-01,3.151219e-02,1.0,0.053158,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mix_IT,0.004988,1.478873e-04,1.070303e-06,0.0,1.000005,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Other_renewable_IT,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Solar_IT,0.000418,1.237887e-05,8.958943e-08,0.0,0.083705,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Waste_IT,0.000005,1.539660e-07,1.114296e-09,0.0,0.001041,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Wind_Offshore_IT,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Build function to condense and visualize data

In [5]:
import numpy as np # need numpy functions 
import pandas as pd # need pandas functions

In [6]:
def group_by_family(matrix):
    families = {'Fossil':['Fossil_Brown_coal/Lignite', 'Fossil_Coal-derived_gas',
                          'Fossil_Gas', 'Fossil_Hard_coal', 'Fossil_Oil', 'Fossil_Oil_shale',
                          'Fossil_Peat','Other_fossil'],
                'Nuclear':['Nuclear'],
                'Hydro':['Hydro_Pumped_Storage','Hydro_Run-of-river_and_poundage',
                         'Hydro_Water_Reservoir', 'Marine'],
                'Other Renwable':['Biomass', 'Geothermal', 'Other_renewable',
                                  'Solar', 'Waste', 'Wind_Offshore', 'Wind_Onshore'],
                'External imports':['Mix']}
    
    per_family = []
    for f in families:
        idx = [k for k in matrix.index if "_".join(k.split("_")[:-1]) in families[f]]
        per_family.append(pd.Series(matrix.loc[idx].sum(), name=f))
    return pd.concat(per_family, axis=1)


def group_per_country(matrix):
    """Function to group datasets per country"""
    countries = np.unique([k.split("_")[-1] for k in matrix.index])
    
    per_country = []
    for c in countries:
        idx = [k for k in matrix.index if k.split("_")[-1]==c]
        per_country.append(pd.Series(matrix.loc[idx].sum(), name=c))
        
    return pd.concat(per_country,axis=1)

## Visualize the origin per production type (columns) in each country (index)

In [7]:
# Visualize table grouped by type of plant for 1st time step
group_by_family(mix[0].loc["Mix_Other":, :"Mix_IT"])

,Fossil,Nuclear,Hydro,Other Renwable,External imports
Mix_AT,0.485921,0.030766,0.261810,0.097588,0.123915
Mix_CH,0.303191,0.415665,0.141009,0.135522,0.004613
Mix_DE,0.593127,0.136792,0.030061,0.237861,0.002158
Mix_FR,0.126312,0.713809,0.095752,0.064128,0.000000
Mix_IT,0.666016,0.046303,0.090563,0.196964,0.000154


In [8]:
# Visualize table grouped by type of plant for 2st time step
group_by_family(mix[1].loc["Mix_Other":, :"Mix_IT"])

,Fossil,Nuclear,Hydro,Other Renwable,External imports
Mix_AT,0.504558,0.033226,0.234255,0.097251,0.130711
Mix_CH,0.291884,0.413615,0.163239,0.129777,0.001485
Mix_DE,0.594143,0.141349,0.028476,0.233432,0.002601
Mix_FR,0.130971,0.713597,0.089035,0.066397,0.000000
Mix_IT,0.659795,0.051923,0.104881,0.183339,0.000063


## Visualize the origin per country (columns) in each country (index)

In [9]:
# Visualize table grouped by country for 1st time step
group_per_country(mix[0].loc["Mix_Other":, :"Mix_IT"]) # 1st time step

,AT,CH,DE,FR,IT,Other
Mix_AT,0.648394,0.000558,0.215238,0.007299,4.594943e-03,0.123915
Mix_CH,0.019224,0.313375,0.431122,0.231531,1.362325e-04,0.004613
Mix_DE,0.000139,0.002268,0.963922,0.031512,9.859542e-07,0.002158
Mix_FR,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000
Mix_IT,0.000642,0.010460,0.014390,0.053158,9.211959e-01,0.000154


In [10]:
# Visualize table grouped by country for 2nd time step
group_per_country(mix[1].loc["Mix_Other":, :"Mix_IT"]) # 2nd time step

,AT,CH,DE,FR,IT,Other
Mix_AT,0.636863,0.001394,0.218425,0.009160,3.447687e-03,0.130711
Mix_CH,0.001698,0.327280,0.419678,0.249849,9.194443e-06,0.001485
Mix_DE,0.000013,0.002588,0.957950,0.036848,7.269581e-08,0.002601
Mix_FR,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000
Mix_IT,0.000073,0.013993,0.017943,0.058663,9.092658e-01,0.000063
